In [2]:
import pandas as pd
from datetime import datetime

In [36]:
data = pd.read_csv("./green_tripdata_2019-09.csv")
data["lpep_pickup_datetime"] = pd.to_datetime(data["lpep_pickup_datetime"])
data["lpep_dropoff_datetime"] = pd.to_datetime(data["lpep_dropoff_datetime"])
data.head()


/tmp/ipykernel_90704/3505600359.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("./green_tripdata_2019-09.csv")


,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,2.0,2019-09-01 00:10:53,2019-09-01 00:23:46,N,1.0,65,189,5.0,2.00,10.5,0.5,0.5,2.36,0.0,NaN,0.3,14.16,1.0,1.0,0.0
1,2.0,2019-09-01 00:31:22,2019-09-01 00:44:37,N,1.0,97,225,5.0,3.20,12.0,0.5,0.5,0.00,0.0,NaN,0.3,13.30,2.0,1.0,0.0
2,2.0,2019-09-01 00:50:24,2019-09-01 01:03:20,N,1.0,37,61,5.0,2.99,12.0,0.5,0.5,0.00,0.0,NaN,0.3,13.30,2.0,1.0,0.0
3,2.0,2019-09-01 00:27:06,2019-09-01 00:33:22,N,1.0,145,112,1.0,1.73,7.5,0.5,0.5,1.50,0.0,NaN,0.3,10.30,1.0,1.0,0.0
4,2.0,2019-09-01 00:43:23,2019-09-01 00:59:54,N,1.0,112,198,1.0,3.42,14.0,0.5,0.5,3.06,0.0,NaN,0.3,18.36,1.0,1.0,0.0


In [37]:
zone_data = pd.read_csv("./taxi_zone_lookup.csv")
zone_data.head()

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone


### Question 3. Count Record

In [4]:
specific_date = datetime.strptime("2019-09-18", "%Y-%m-%d").date()
# print(data["lpep_pickup_datetime"].dt.date)

filter = ((data["lpep_pickup_datetime"].dt.date == specific_date) & (data["lpep_dropoff_datetime"].dt.date == specific_date))
filtered_data = data[filter]
print(len(filtered_data))

15612


### Question 4. Longest trip for each day

In [5]:
longest_trip = data["trip_distance"].max()
print(longest_trip)

data["pickup_datetime"] = data["lpep_pickup_datetime"].dt.date
groupby_data = data.groupby(["pickup_datetime"])["trip_distance"].max().reset_index()

# print(groupby_data)
print(groupby_data[groupby_data["trip_distance"] == longest_trip])

341.64
   pickup_datetime  trip_distance
29      2019-09-26         341.64


### Question 5. Three biggest pick up Boroughs

In [38]:
full_data = data.merge(zone_data, left_on='PULocationID', right_on="LocationID")
full_data.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,LocationID,Borough,Zone,service_zone
0,2.0,2019-09-01 00:10:53,2019-09-01 00:23:46,N,1.0,65,189,5.0,2.00,10.5,...,NaN,0.3,14.16,1.0,1.0,0.0,65,Brooklyn,Downtown Brooklyn/MetroTech,Boro Zone
1,2.0,2019-09-01 00:31:22,2019-09-01 00:44:37,N,1.0,97,225,5.0,3.20,12.0,...,NaN,0.3,13.30,2.0,1.0,0.0,97,Brooklyn,Fort Greene,Boro Zone
2,2.0,2019-09-01 00:50:24,2019-09-01 01:03:20,N,1.0,37,61,5.0,2.99,12.0,...,NaN,0.3,13.30,2.0,1.0,0.0,37,Brooklyn,Bushwick South,Boro Zone
3,2.0,2019-09-01 00:27:06,2019-09-01 00:33:22,N,1.0,145,112,1.0,1.73,7.5,...,NaN,0.3,10.30,1.0,1.0,0.0,145,Queens,Long Island City/Hunters Point,Boro Zone
4,2.0,2019-09-01 00:43:23,2019-09-01 00:59:54,N,1.0,112,198,1.0,3.42,14.0,...,NaN,0.3,18.36,1.0,1.0,0.0,112,Brooklyn,Greenpoint,Boro Zone


In [28]:
# Find top 3 pickup location
groupby_data = full_data.groupby(["Borough"])["total_amount"].sum().reset_index().sort_values("total_amount", ascending=False)
print(groupby_data.head())
top_3_location_id = list(groupby_data.iloc[:3,]["Borough"])
print(top_3_location_id)

     Borough  total_amount
1   Brooklyn    2619378.54
4     Queens    2460386.17
3  Manhattan    2427880.92
0      Bronx     818158.06
6    Unknown      25852.58
['Brooklyn', 'Queens', 'Manhattan']


### Question 6. Largest tip

In [54]:
zone_data_renamed = zone_data.rename(columns=lambda x: x + "_pu")
partial_df = data.merge(zone_data_renamed, left_on='PULocationID', right_on="LocationID_pu")
partial_df.columns

Index(['VendorID', 'lpep_pickup_datetime', 'lpep_dropoff_datetime',
       'store_and_fwd_flag', 'RatecodeID', 'PULocationID', 'DOLocationID',
       'passenger_count', 'trip_distance', 'fare_amount', 'extra', 'mta_tax',
       'tip_amount', 'tolls_amount', 'ehail_fee', 'improvement_surcharge',
       'total_amount', 'payment_type', 'trip_type', 'congestion_surcharge',
       'LocationID_pu', 'Borough_pu', 'Zone_pu', 'service_zone_pu'],
      dtype='object')

In [62]:
zone_data_renamed = zone_data.rename(columns=lambda x: x + "_do")
full_df = partial_df.merge(zone_data_renamed, left_on='DOLocationID', right_on="LocationID_do")

filter = (full_df["lpep_pickup_datetime"].dt.year == 2019) & (full_df["lpep_pickup_datetime"].dt.month == 9) & (full_df["Zone_pu"] == "Astoria")
full_df[filter].groupby("Zone_do")["tip_amount"].max().reset_index().sort_values("tip_amount", ascending=False).head(1)

,Zone_do,tip_amount
98,JFK Airport,62.31
